## 0. Dependencies

In [1]:
!pip install mediapipe opencv-python pandas scikit-learn

     --------------------------------------- 49.8/49.8 MB 25.2 MB/s eta 0:00:00
     --------------------------------------- 35.6/35.6 MB 24.2 MB/s eta 0:00:00
     --------------------------------------- 42.5/42.5 MB 23.4 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.5.0rc0 requires flatbuffers~=1.12.0, but you have flatbuffers 22.12.6 which is incompatible.
tensorflow 2.5.0rc0 requires numpy~=1.19.2, but you have numpy 1.23.4 which is incompatible.

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
import mediapipe as mp
import cv2
import numpy as np
import time

In [26]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

## 1. Detections

In [27]:
cap = cv2.VideoCapture(0)

# model initiation
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, image = cap.read()
        
        # Feed color conversion
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # pose detections
        results = pose.process(image)
        
        # revert back to bgr for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
        
        mp_drawing.draw_landmarks(image,results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                                  ,mp_drawing.DrawingSpec(color=(245,117,66),
                                                          thickness=2, circle_radius=4)
                                  ,mp_drawing.DrawingSpec(color=(245,66,230),
                                                          thickness=2,circle_radius=2))
        cv2.imshow('Raw Feed',image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()
    

## 1.1 Video for Training

In [5]:
cap =cv2.VideoCapture(0)

height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
fps = cap.get(cv2.CAP_PROP_FPS)
vwriter = cv2.VideoWriter('train.avi',cv2.VideoWriter_fourcc('P','I','M','1'), 
                          fps, (int(width), int(height)))
time.sleep(5)
while cap.isOpened():
    ret,frame = cap.read()
    
    try: 
        cv2.imshow("Training Video",frame)
        vwriter.write(frame)
    except Exception as e:
        break
        
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
vwriter.release()
cv2.destroyAllWindows()

## 2. Extracting coordinates

In [6]:
import csv
import os
import numpy as np
from matplotlib import pyplot as plt

In [9]:
landmarks = ['class']
for val in range(1,34):
    landmarks += ['x{}'.format(val),'y{}'.format(val),'z{}'.format(val),'v{}'.format(val)]

In [10]:
landmarks

['class',
 'x1',
 'y1',
 'z1',
 'v1',
 'x2',
 'y2',
 'z2',
 'v2',
 'x3',
 'y3',
 'z3',
 'v3',
 'x4',
 'y4',
 'z4',
 'v4',
 'x5',
 'y5',
 'z5',
 'v5',
 'x6',
 'y6',
 'z6',
 'v6',
 'x7',
 'y7',
 'z7',
 'v7',
 'x8',
 'y8',
 'z8',
 'v8',
 'x9',
 'y9',
 'z9',
 'v9',
 'x10',
 'y10',
 'z10',
 'v10',
 'x11',
 'y11',
 'z11',
 'v11',
 'x12',
 'y12',
 'z12',
 'v12',
 'x13',
 'y13',
 'z13',
 'v13',
 'x14',
 'y14',
 'z14',
 'v14',
 'x15',
 'y15',
 'z15',
 'v15',
 'x16',
 'y16',
 'z16',
 'v16',
 'x17',
 'y17',
 'z17',
 'v17',
 'x18',
 'y18',
 'z18',
 'v18',
 'x19',
 'y19',
 'z19',
 'v19',
 'x20',
 'y20',
 'z20',
 'v20',
 'x21',
 'y21',
 'z21',
 'v21',
 'x22',
 'y22',
 'z22',
 'v22',
 'x23',
 'y23',
 'z23',
 'v23',
 'x24',
 'y24',
 'z24',
 'v24',
 'x25',
 'y25',
 'z25',
 'v25',
 'x26',
 'y26',
 'z26',
 'v26',
 'x27',
 'y27',
 'z27',
 'v27',
 'x28',
 'y28',
 'z28',
 'v28',
 'x29',
 'y29',
 'z29',
 'v29',
 'x30',
 'y30',
 'z30',
 'v30',
 'x31',
 'y31',
 'z31',
 'v31',
 'x32',
 'y32',
 'z32',
 'v32',
 '

In [12]:
with open("coords.csv",mode='w',newline='') as f:
    csv_writer = csv.writer(f,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [7]:
def export_landmark(results,action):
    try:
        keypoints = np.array([[res.x,res.y,
                        res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
        keypoints.insert(0,action)
        
        with open('coords.csv',mode='a',newline='') as f:
            csv_writer = csv.writer(f,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
    except Exception as e:
        print(e)

In [27]:
export_landmark(results,'up')

In [23]:
results.pose_landmarks.landmark

[x: 0.56568074
y: 0.51079047
z: -0.2573844
visibility: 0.9976786
, x: 0.5690486
y: 0.5091757
z: -0.2681176
visibility: 0.9964705
, x: 0.5781413
y: 0.5073992
z: -0.26813167
visibility: 0.99610615
, x: 0.5805404
y: 0.5065457
z: -0.26823512
visibility: 0.9966863
, x: 0.56867486
y: 0.5082597
z: -0.2680493
visibility: 0.9946927
, x: 0.5663357
y: 0.50725937
z: -0.26806474
visibility: 0.99341375
, x: 0.56399465
y: 0.50626993
z: -0.26813492
visibility: 0.9938545
, x: 0.58526
y: 0.4947111
z: -0.24636266
visibility: 0.99467045
, x: 0.5597103
y: 0.49380943
z: -0.24606809
visibility: 0.98967195
, x: 0.57505614
y: 0.5155839
z: -0.24056557
visibility: 0.99767965
, x: 0.5678863
y: 0.5153179
z: -0.24077317
visibility: 0.9968576
, x: 0.6101073
y: 0.50473046
z: -0.19409625
visibility: 0.99909073
, x: 0.5291463
y: 0.49964124
z: -0.18850705
visibility: 0.9984703
, x: 0.6201933
y: 0.5919916
z: -0.16760233
visibility: 0.8012985
, x: 0.51315373
y: 0.57876134
z: -0.16635193
visibility: 0.6375928
, x: 0.628794

In [28]:
cap = cv2.VideoCapture('train.avi')
#Initiate model
with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
    while cap.isOpened():
        ret,frame = cap.read()
        
        #recolor
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        #Make Detections
        results = pose.process(image)
        
        #Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
        
        mp_drawing.draw_landmarks(image,results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                                  ,mp_drawing.DrawingSpec(color=(245,117,66),
                                                          thickness=2, circle_radius=4)
                                  ,mp_drawing.DrawingSpec(color=(245,66,230),
                                                          thickness=2,circle_radius=2))
        k = cv2.waitKey(1)
        if k == 117:
            export_landmark(results,'up')
        if k == 100:
            export_landmark(results,'down')
        
        cv2.imshow('Raw Webcam Feed',image)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break;
cap.release()
cv2.destroyAllWindows()

## 3. Training 

In [4]:
from sklearn.model_selection import train_test_split
import pandas as pd

In [5]:
df = pd.read_csv('coords.csv')

In [6]:
df

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,up,0.346983,0.194615,-0.347844,0.999942,0.351684,0.179568,-0.327277,0.999847,0.355946,...,0.181894,0.322809,0.425807,0.936631,0.039865,0.507773,0.292817,0.949712,0.022930,0.529336
1,up,0.378101,0.181778,-0.380143,0.999961,0.384015,0.167068,-0.361462,0.999893,0.388751,...,0.243924,0.316235,0.431627,0.910798,0.164356,0.466815,0.342254,0.930429,0.097213,0.511925
2,up,0.378656,0.181531,-0.386943,0.999963,0.383850,0.167292,-0.368829,0.999899,0.388620,...,0.216723,0.330974,0.429742,0.895260,0.200371,0.474826,0.336920,0.926098,0.069455,0.528580
3,up,0.375709,0.181124,-0.364818,0.999967,0.381624,0.166921,-0.347258,0.999909,0.386387,...,0.242863,0.358080,0.423580,0.887065,0.204066,0.494882,0.337708,0.918308,0.101864,0.554229
4,up,0.375341,0.180242,-0.396656,0.999968,0.381495,0.166417,-0.377608,0.999909,0.386345,...,0.247261,0.369687,0.421358,0.876903,0.194823,0.509407,0.342654,0.905193,0.104222,0.567244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,up,0.368123,0.175331,-0.340049,0.999807,0.371984,0.165343,-0.316321,0.999591,0.375674,...,0.218180,0.325081,0.418255,0.899166,0.151717,0.459554,0.339203,0.928920,0.060121,0.505008
184,up,0.368928,0.175741,-0.335065,0.999846,0.372440,0.165904,-0.311542,0.999665,0.376070,...,0.201505,0.350068,0.422223,0.891880,0.158166,0.490343,0.339214,0.925834,0.039825,0.539814
185,up,0.368342,0.177300,-0.331730,0.999872,0.371984,0.167329,-0.308499,0.999712,0.375657,...,0.195004,0.362273,0.422702,0.913300,0.127884,0.509733,0.342218,0.941711,0.030309,0.559529
186,up,0.364108,0.177521,-0.338985,0.999887,0.367215,0.167087,-0.316309,0.999740,0.371164,...,0.202529,0.357882,0.424653,0.916504,0.113736,0.501108,0.345095,0.941931,0.039053,0.551804


In [7]:
x = df.drop('class',axis = 1)
y = df['class']

In [8]:
x

,x1,y1,z1,v1,x2,y2,z2,v2,x3,y3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,0.346983,0.194615,-0.347844,0.999942,0.351684,0.179568,-0.327277,0.999847,0.355946,0.179012,...,0.181894,0.322809,0.425807,0.936631,0.039865,0.507773,0.292817,0.949712,0.022930,0.529336
1,0.378101,0.181778,-0.380143,0.999961,0.384015,0.167068,-0.361462,0.999893,0.388751,0.167063,...,0.243924,0.316235,0.431627,0.910798,0.164356,0.466815,0.342254,0.930429,0.097213,0.511925
2,0.378656,0.181531,-0.386943,0.999963,0.383850,0.167292,-0.368829,0.999899,0.388620,0.167271,...,0.216723,0.330974,0.429742,0.895260,0.200371,0.474826,0.336920,0.926098,0.069455,0.528580
3,0.375709,0.181124,-0.364818,0.999967,0.381624,0.166921,-0.347258,0.999909,0.386387,0.166968,...,0.242863,0.358080,0.423580,0.887065,0.204066,0.494882,0.337708,0.918308,0.101864,0.554229
4,0.375341,0.180242,-0.396656,0.999968,0.381495,0.166417,-0.377608,0.999909,0.386345,0.166573,...,0.247261,0.369687,0.421358,0.876903,0.194823,0.509407,0.342654,0.905193,0.104222,0.567244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,0.368123,0.175331,-0.340049,0.999807,0.371984,0.165343,-0.316321,0.999591,0.375674,0.165585,...,0.218180,0.325081,0.418255,0.899166,0.151717,0.459554,0.339203,0.928920,0.060121,0.505008
184,0.368928,0.175741,-0.335065,0.999846,0.372440,0.165904,-0.311542,0.999665,0.376070,0.166364,...,0.201505,0.350068,0.422223,0.891880,0.158166,0.490343,0.339214,0.925834,0.039825,0.539814
185,0.368342,0.177300,-0.331730,0.999872,0.371984,0.167329,-0.308499,0.999712,0.375657,0.167917,...,0.195004,0.362273,0.422702,0.913300,0.127884,0.509733,0.342218,0.941711,0.030309,0.559529
186,0.364108,0.177521,-0.338985,0.999887,0.367215,0.167087,-0.316309,0.999740,0.371164,0.167372,...,0.202529,0.357882,0.424653,0.916504,0.113736,0.501108,0.345095,0.941931,0.039053,0.551804


In [9]:
y

0      up
1      up
2      up
3      up
4      up
       ..
183    up
184    up
185    up
186    up
187    up
Name: class, Length: 188, dtype: object

In [10]:
x_train , x_test, y_train, y_test = train_test_split(x,y,test_size = 0.3,random_state = 1234)

In [11]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression , RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [12]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [14]:
fit_model = {}
for algo , pipeline in pipelines.items():
    model = pipeline.fit(x_train,y_train)
    fit_model[algo] = model


In [15]:
fit_model['rc'].predict(x_test)

array(['up', 'down', 'up', 'down', 'up', 'up', 'down', 'up', 'down', 'up',
       'up', 'up', 'up', 'up', 'up', 'down', 'up', 'up', 'up', 'up', 'up',
       'up', 'down', 'down', 'down', 'up', 'down', 'up', 'up', 'down',
       'up', 'up', 'down', 'up', 'up', 'up', 'down', 'up', 'down', 'down',
       'down', 'up', 'down', 'up', 'down', 'up', 'up', 'up', 'down',
       'down', 'down', 'up', 'down', 'down', 'up', 'down', 'up'],
      dtype='<U4')

In [17]:
y_test

83       up
54     down
63       up
175    down
136    down
146      up
144    down
29       up
31     down
117      up
132      up
161      up
95       up
48       up
168      up
137    down
51       up
6        up
114      up
170      up
5        up
166      up
40     down
141    down
104    down
7        up
122    down
65       up
9        up
92     down
148      up
149      up
110    down
101      up
185      up
66       up
59     down
43       up
93     down
121    down
39     down
25       up
125    down
130      up
94     down
17       up
11       up
97       up
123    down
88     down
160    down
119      up
108    down
155    down
27       up
58     down
183      up
Name: class, dtype: object

In [19]:
from sklearn.metrics import accuracy_score , precision_score , recall_score
import pickle

In [21]:
for algo,model in fit_model.items():
    yhat = model.predict(x_test)
    print(algo,accuracy_score(y_test.values,yhat),
         precision_score(y_test.values,yhat,average = "binary",pos_label = "up"),
         recall_score(y_test.values,yhat,average = "binary",pos_label = "up"))

lr 0.9824561403508771 0.9705882352941176 1.0
rc 0.9824561403508771 0.9705882352941176 1.0
rf 1.0 1.0 1.0
gb 1.0 1.0 1.0


In [23]:
yhat = fit_model['rf'].predict(x_test)
yhat[:10]

array(['up', 'down', 'up', 'down', 'down', 'up', 'down', 'up', 'down',
       'up'], dtype=object)

In [30]:
with open('model.pkl','wb') as f:
    pickle.dump(fit_model['rf'],f)

In [31]:
with open('model.pkl','rb') as f:
    model = pickle.load(f)

In [32]:
model.predict(x_test)

array(['up', 'down', 'up', 'down', 'down', 'up', 'down', 'up', 'down',
       'up', 'up', 'up', 'up', 'up', 'up', 'down', 'up', 'up', 'up', 'up',
       'up', 'up', 'down', 'down', 'down', 'up', 'down', 'up', 'up',
       'down', 'up', 'up', 'down', 'up', 'up', 'up', 'down', 'up', 'down',
       'down', 'down', 'up', 'down', 'up', 'down', 'up', 'up', 'up',
       'down', 'down', 'down', 'up', 'down', 'down', 'up', 'down', 'up'],
      dtype=object)